In [1]:
import pygame
import sys
import random
import cv2
import time
import mediapipe as mp

# Initialize Pygame
pygame.init()

# Constants
SCREEN_WIDTH, SCREEN_HEIGHT = 800, 480
BIRD_WIDTH, BIRD_HEIGHT = 80, 80
PIPE_WIDTH = 120
PIPE_GAP = 200
GRAVITY = 0.3
# BIRD_JUMP = -8
FPS = 60
PIPE_SPEED = 4
ACCELERATION = 0.01
PIPE_VARIATION = 1

# Colors
WHITE = (255, 255, 255)
GREEN = (0, 255, 0)
BLACK = (0, 0, 0)
YELLOW = (253, 218, 13)

# Create the game window
screen = pygame.display.set_mode((SCREEN_WIDTH, SCREEN_HEIGHT), pygame.RESIZABLE)
pygame.display.set_caption("Flappy Bird")

# Function to display intro screen
def display_intro():
    duration = 10000
    fade_duration = 3000
    intro_image = pygame.image.load("intro_image.jpg")
    intro_image = pygame.transform.scale(intro_image, (SCREEN_WIDTH, SCREEN_HEIGHT))
    screen.blit(intro_image, (0, 0))
    pygame.display.flip()

    # Get the current time
    start_time = pygame.time.get_ticks()

    while pygame.time.get_ticks() - start_time < duration:
        # Calculate the alpha value for fading
        alpha = min(255, (pygame.time.get_ticks() - start_time) /  fade_duration * 255)

        # Create a surface with per-pixel alpha
        fade_surface = pygame.Surface((SCREEN_WIDTH, SCREEN_HEIGHT), pygame.SRCALPHA)
        pygame.draw.rect(fade_surface, (0, 0, 0, alpha), (0, 0, SCREEN_WIDTH, SCREEN_HEIGHT))
        screen.blit(intro_image, (0, 0))
        screen.blit(fade_surface, (0, 0))
        pygame.display.flip()

    # Clear the screen
    screen.fill((255, 255, 255))
    pygame.display.flip()

    
display_intro()

# Load bird image
bird_img = pygame.image.load("bird2.png")
bird_img = pygame.transform.scale(bird_img, (BIRD_WIDTH, BIRD_HEIGHT))
bird_rect = bird_img.get_rect()
bird_x = 100
bird_y = SCREEN_HEIGHT // 2

# Create a list to store pipe positions
pipes = []

# Score
score = 0
high_score = 0  # Initialize high score

# Create a clock object to control the frame rate
clock = pygame.time.Clock()

# Bird velocity
bird_velocity = 0.0  # Initial velocity

# Function to generate pipes
def create_pipe():
    global PIPE_GAP
    if PIPE_GAP>=130:
        PIPE_GAP-=PIPE_VARIATION
    pipe_height = random.randint(50, SCREEN_HEIGHT - PIPE_GAP - 50)
    top_pipe = pygame.Rect(SCREEN_WIDTH, 0, PIPE_WIDTH, pipe_height)
    bottom_pipe = pygame.Rect(SCREEN_WIDTH, pipe_height + PIPE_GAP, PIPE_WIDTH, SCREEN_HEIGHT - pipe_height - PIPE_GAP)
    return top_pipe, bottom_pipe

cap = cv2.VideoCapture(0)
mpHands = mp.solutions.hands
hands = mpHands.Hands()
mpDraw = mp.solutions.drawing_utils

# Function to reset the game
def reset_game():
    global bird_y, pipes, score, high_score, PIPE_GAP,PIPE_SPEED, game_over, counting_down
    bird_y = SCREEN_HEIGHT // 2
    pipes = []
    score = 0
    if score > high_score:
        high_score = score
    PIPE_GAP = 200
    bird_y = SCREEN_HEIGHT // 2
    PIPE_SPEED = 4
    game_over = False
    counting_down = True
    PIPE_GAP=200

# Function for loading screen
def countdown():
    font = pygame.font.Font(None, 72)
    for i in range(100, 0, -1):
        screen.fill(WHITE)
        countdown_text = font.render(f"Loading...{i}", True, GREEN)
        screen.blit(countdown_text, (SCREEN_WIDTH // 2 - 140, SCREEN_HEIGHT // 2 - 20))
        pygame.display.flip()
        pygame.time.delay(55)
        



# Main game loop
running = True
game_over = False
counting_down = True
game_started = False

while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False
        elif event.type == pygame.KEYDOWN:
            if not game_started:
                if event.key == pygame.K_SPACE:
                    game_started = True
            elif game_over:
                if event.key == pygame.K_SPACE:
                    reset_game()
                elif event.key == pygame.K_ESCAPE:
                    running = False

    success, image = cap.read()
    imageRGB = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(imageRGB)

    if results.multi_hand_landmarks:
        for handLms in results.multi_hand_landmarks:
            for id, lm in enumerate(handLms.landmark):
                h, w, c = image.shape
                cx, cy = int(lm.x * w), int(lm.y * h)
                bird_y = cy
    else:
        bird_y += GRAVITY
        bird_rect.topleft = (bird_x, bird_y)

    if counting_down:
        countdown()
        counting_down = False

    if not game_over:
        # Move the bird
        bird_rect.topleft = (bird_x, bird_y)

        # Generate pipes
        if len(pipes) == 0 or pipes[-1].left < SCREEN_WIDTH - 300:
            pipes.extend(create_pipe())

        # Move pipes
        for pipe in pipes:
            if score%5==0:
                PIPE_SPEED += ACCELERATION
            pipe.left -= PIPE_SPEED

        # Remove off-screen pipes
        pipes = [pipe for pipe in pipes if pipe.right > 0]

        # Check for collisions
        for pipe in pipes:
            if bird_rect.colliderect(pipe):
                game_over = True

        # Check for scoring
        if pipes and bird_x > pipes[0].right:
            score += 1
            pipes.pop(0)

        # Check if the bird hits the ground
        if bird_y >= SCREEN_HEIGHT - BIRD_HEIGHT:
            game_over = True

    # Clear the screen
    screen.fill(WHITE)
    
    if not game_started:
        # Display start screen
        font = pygame.font.Font(None, 72)
        start_text = font.render(" SPACE to START ", True, BLACK)
        screen.blit(start_text, (SCREEN_WIDTH // 2 - 220, SCREEN_HEIGHT // 2 - 20))
    else:
        # Draw pipes
        for pipe in pipes:
            pygame.draw.rect(screen, GREEN, pipe)
        # Draw bird
        screen.blit(bird_img, bird_rect)
        if game_over:
            # Display game over message
            font = pygame.font.Font(None, 72)
            game_over_text = font.render("Game Over", True, YELLOW)
            screen.blit(game_over_text, (SCREEN_WIDTH // 2 - 150, SCREEN_HEIGHT // 2 - 50))

            font = pygame.font.Font(None, 36)
            score_text = font.render(f"Score: {score}", True, BLACK)
            screen.blit(score_text, (SCREEN_WIDTH // 2 - 50, SCREEN_HEIGHT // 2 + 20))

            if score > high_score:
                high_score = score
                high_score_text = font.render(f"New High Score: {high_score}", True, YELLOW)
                screen.blit(high_score_text, (SCREEN_WIDTH // 2 - 100, SCREEN_HEIGHT // 2 + 60))
            else:
                high_score_text = font.render(f"High Score: {high_score}", True, YELLOW)
                screen.blit(high_score_text, (SCREEN_WIDTH // 2 - 60, SCREEN_HEIGHT // 2 + 60))
        # Display play again message
            play_again_text = font.render("Press SPACE to Play Again", True, BLACK)
            screen.blit(play_again_text, (SCREEN_WIDTH // 2 - 180, SCREEN_HEIGHT // 2 + 100))
        else:
        # Draw score during the game
            font = pygame.font.Font(None, 36)
            score_text = font.render(f"Score: {score}", True, BLACK)
            screen.blit(score_text, (10, 10))

    pygame.display.flip()
    clock.tick(FPS)

# Quit the game
pygame.quit()
sys.exit()



pygame 2.5.0 (SDL 2.28.0, Python 3.11.5)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

C:\Users\Admin\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3534: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
